In [ ]:
import json
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Found existing installation: unsloth 2024.11.11
Uninstalling unsloth-2024.11.11:
  Successfully uninstalled unsloth-2024.11.11
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-z7zya9dy
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-z7zya9dy
  Resolved https://github.com/unslothai/unsloth.git to commit 8558bc92b06f9128499484ef737fa71b966ffc23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.10-py3-none-any.whl size=166794 sha256=c1672f3e5c8fe0a7c2155d32df71e07fd995906d5f49ed149b30533f88c74c0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-lcid7ayl/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install python-multipart bitsandbytes

In [ ]:
import os
import requests
import json
import base64
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import nltk
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from fastapi import FastAPI, HTTPException, Form
from pydantic import BaseModel
import uvicorn
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from pyngrok import ngrok
import threading
import nest_asyncio
import uuid

Mounted at /content/gdrive


In [ ]:
# API Keys
os.environ["GOOEY_API_KEY"] = "your-gooey-api-key"
os.environ["STABILITY_API_KEY"] = "your-stability-api-key"

# Download NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# System instruction
system_instruction = """You are a kind, friendly, and patient assistant specifically designed to help autistic and dyslexic children. Your primary goal is to nurture their communication and social skills through meaningful and engaging conversations."""

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Function to generate model response
def generate_response(user_input, conversation_history, tokenizer, model):
    # Add user input to conversation history
    conversation_history.append({"role": "user", "content": user_input})

    # Format the conversation for the model
    inputs = tokenizer.apply_chat_template(
        conversation_history,
        tokenize=True,
        add_generation_prompt=True,  # Required for generation
        return_tensors="pt",
    ).to("cuda")

    # Generate response
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=128,
        use_cache=True,
        temperature=1.0,
        min_p=0.1,
    )

    # Decode model response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    assistant_response = response.split("assistant")[-1].strip()

    # Add assistant response to conversation history
    conversation_history.append({"role": "assistant", "content": assistant_response})

    return assistant_response

==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Unsloth 2024.11.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# Function to extract nouns and adjectives from the chatbot response
def extract_nouns_adjectives(text):
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    nouns_adjectives = [word for word, pos in pos_tags if pos in ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS']]
    return nouns_adjectives

# Function to generate an image prompt from extracted nouns and adjectives
def generate_image_prompt(nouns_adjectives):
    if not nouns_adjectives:
        return None  # No image needed
    prompt = "A visual representation of: " + ', '.join(nouns_adjectives)
    return prompt

# Function to generate an image using Stability AI API
def generate_image_from_prompt(prompt, output_image_path, width=512, height=512):
    if not prompt:
        return None

    engine_id = "stable-diffusion-v1-6"
    api_host = os.getenv('API_HOST', 'https://api.stability.ai')
    api_key = os.getenv("STABILITY_API_KEY")

    if api_key is None:
        raise Exception("Missing Stability API key. Please set the 'STABILITY_API_KEY' environment variable.")

    # Send the request to Stability AI API
    response = requests.post(
        f"{api_host}/v1/generation/{engine_id}/text-to-image",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {api_key}"
        },
        json={
            "text_prompts": [
                {
                    "text": prompt  # Using the prompt generated from chatbot
                }
            ],
            "cfg_scale": 7,
            "height": height,
            "width": width,
            "samples": 1,
            "steps": 30,
        },
    )

    # Check for errors
    if response.status_code != 200:
        raise Exception("Non-200 response from Stability AI: " + str(response.text))

    # Extract the image from the response
    data = response.json()
    for image in data["artifacts"]:
        image_data = base64.b64decode(image["base64"])
        with open(output_image_path, 'wb') as f:
            f.write(image_data)
        return Image.open(BytesIO(image_data))

In [ ]:
# Load fine-tuned chatbot model from Google Drive
model_path = "/content/drive/MyDrive/lora_model_llama3.2"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Step 3: Enable faster inference mode
FastLanguageModel.for_inference(model)

# Define FastAPI app
app = FastAPI()

# Add CORS middleware to avoid cross-origin issues
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # You can specify certain origins instead of "*"
    allow_credentials=True,
    allow_methods=["*"],  # Allow all methods (GET, POST, etc.)
    allow_headers=["*"],  # Allow all headers
)

# Define input and output models
class UserInput(BaseModel):
    user_input: str


class APIResponse(BaseModel):
    response: str
    image_url: str = None  # Path to the generated image





In [ ]:
# Endpoint to process the input and return chatbot response and image
@app.post("/process")
async def process_input(user_input: str = Form(...)):
    try:
        # Generate chatbot response
        conversation_history = [{"role": "system", "content": system_instruction}]
        response = generate_response(user_input, conversation_history, tokenizer, model)

        # Extract nouns and adjectives from the response
        nouns_adjectives = extract_nouns_adjectives(response)

        # Generate an image prompt
        image_prompt = generate_image_prompt(nouns_adjectives)

        # Generate image from prompt
        image_url = None
        if image_prompt:
            session_id = str(uuid.uuid4())
            generated_image_path = f"/content/generated_image_{session_id}.png"
            generate_image_from_prompt(image_prompt, generated_image_path)
            image_url = generated_image_path

        # Return the chatbot response and image URL
        return {
            "response": response,
            "image_url": image_url,
        }

    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})




In [ ]:
# Ngrok setup for running in Colab
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:

# Apply nest_asyncio to allow running FastAPI in Colab/Jupyter
nest_asyncio.apply()



In [ ]:
# Start the FastAPI server in a background thread
thread = threading.Thread(target=run_app)
thread.daemon = True
thread.start()



In [ ]:
# Replace with your Ngrok authtoken
NGROK_AUTHTOKEN = "your-ngrok-auth-token"
!ngrok authtoken {NGROK_AUTHTOKEN}



In [ ]:
# Connect Ngrok to the running FastAPI server
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

In [ ]:
!curl -X POST https://542f-35-187-255-30.ngrok-free.app/generate-response \
-H "Content-Type: application/json" \
-d '{"user_input": "Hi"}'

INFO:     35.187.255.30:0 - "POST /generate-response HTTP/1.1" 200 OK
{"response":"Hi Lily! You’re going to learn so much today. Let’s","image":"..."}